<a href="https://colab.research.google.com/github/Roudranil/competitions/blob/main/drivendata/flu%20shot%20learning/flu_shot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
import numpy as np
import pandas as pd

In [18]:
train_x = pd.read_csv('/content/drive/MyDrive/data/drivendata_flu shot /train_x.csv', index_col=0)
train_y = pd.read_csv('/content/drive/MyDrive/data/drivendata_flu shot /train_y.csv', index_col=0)
test = pd.read_csv('/content/drive/MyDrive/data/drivendata_flu shot /test.csv', index_col=0)
submission = pd.read_csv('/content/drive/MyDrive/data/drivendata_flu shot /submission.csv', index_col=0)

In [17]:
submission

,respondent_id,h1n1_vaccine,seasonal_vaccine
0,26707,0.5,0.7
1,26708,0.5,0.7
2,26709,0.5,0.7
3,26710,0.5,0.7
4,26711,0.5,0.7
...,...,...,...
26703,53410,0.5,0.7
26704,53411,0.5,0.7
26705,53412,0.5,0.7
26706,53413,0.5,0.7
